## MLP-based Values prediction 

### 1. Install required dependency libraries

In [35]:
"""
Install required dependency libraries
-pip install pandas
-pip install scikit-learn
-pip install tensorflow
-import matplotlib.pyplot as plt

"""

'\nInstall required dependency libraries\n-pip install pandas\n-pip install scikit-learn\n-pip install tensorflow\n-import matplotlib.pyplot as plt\n\n'

### 2. Import installed required dependency libraries

In [36]:
# Import required libraries for LSTM model
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

### 3. Prepare the datset

In [37]:
# Load the CSV file into a pandas dataframe
dataframe = pd.read_csv("FX_EURUSD.csv")

In [38]:
#To see the whole inforamtion contenet of the above dataset:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184233 entries, 0 to 184232
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   time                       184233 non-null  object 
 1   open                       184233 non-null  float64
 2   high                       184233 non-null  float64
 3   low                        184233 non-null  float64
 4   close                      184233 non-null  float64
 5   Plot                       184233 non-null  float64
 6   MA                         184233 non-null  float64
 7   Smoothing Line             184233 non-null  float64
 8   MA.1                       184233 non-null  float64
 9   Smoothing Line.1           184233 non-null  float64
 10  MA.2                       184233 non-null  float64
 11  Smoothing Line.2           184233 non-null  float64
 12  Rating                     184233 non-null  float64
 13  %K                         18

In [39]:
# To see the first five data content of the above dataset:
dataframe.head()

,time,open,high,low,close,Plot,MA,Smoothing Line,MA.1,Smoothing Line.1,...,Aroon Down,ADR,ADX,Bollinger Bands %B,ATR,Plot.1,Plot.2,Bollinger Bands Width,BBPower,MF
0,2015-11-04T04:30:00Z,1.09472,1.09495,1.09462,1.09465,1.094886,1.095562,1.095559,1.095109,1.095276,...,50.000000,0.000539,20.222071,0.327029,0.000563,-0.212121,-0.000600,0.002423,-0.000308,-0.132298
1,2015-11-04T04:45:00Z,1.09465,1.09497,1.09465,1.09496,1.094824,1.095554,1.095560,1.095049,1.095204,...,42.857143,0.000504,19.940993,0.464168,0.000546,0.968750,-0.000597,0.002256,-0.000264,-0.082009
2,2015-11-04T05:00:00Z,1.09496,1.09561,1.09496,1.09531,1.094924,1.095569,1.095562,1.094999,1.095130,...,35.714286,0.000523,18.887983,0.641031,0.000553,0.538462,-0.000566,0.002014,0.000581,-0.105238
3,2015-11-04T05:15:00Z,1.09531,1.09555,1.09512,1.09534,1.095091,1.095586,1.095566,1.094964,1.095064,...,28.571429,0.000501,17.910188,0.685753,0.000545,0.069767,-0.000494,0.001846,0.000582,-0.091719
4,2015-11-04T05:30:00Z,1.09534,1.09561,1.09527,1.09556,1.095279,1.095597,1.095574,1.094925,1.095009,...,21.428571,0.000494,17.135591,0.878842,0.000530,0.647059,-0.000358,0.001531,0.000645,-0.073619


In [40]:
# Drop the rows containing NaN values
dataframe = dataframe.dropna()

In [41]:
# Extract the time and target columns into separate variables
time = dataframe.iloc[:, 0].values
target = dataframe.iloc[:, 2:5].values

In [42]:
# Remove the time and target columns from the original dataframe to get the input data
inputs =dataframe.iloc[:, 5:].values

In [43]:
# Create a new dataframe with only the input data
new_dataframe = pd.DataFrame(inputs)

In [44]:
# Display the new dataframe
print(new_dataframe.head())

         0         1         2         3         4         5         6    
0  1.094886  1.095562  1.095559  1.095109  1.095276  1.099741  1.100111  \
1  1.094824  1.095554  1.095560  1.095049  1.095204  1.099701  1.100073   
2  1.094924  1.095569  1.095562  1.094999  1.095130  1.099665  1.100035   
3  1.095091  1.095586  1.095566  1.094964  1.095064  1.099628  1.099997   
4  1.095279  1.095597  1.095574  1.094925  1.095009  1.099592  1.099959   

         7          8          9   ...         18        19         20   
0 -0.512121  52.631579  47.125741  ...  50.000000  0.000539  20.222071  \
1 -0.109091  70.760234  56.902962  ...  42.857143  0.000504  19.940993   
2  0.090909  83.870968  69.087594  ...  35.714286  0.000523  18.887983   
3 -0.045455  85.483871  80.038358  ...  28.571429  0.000501  17.910188   
4  0.021212  97.311828  88.888889  ...  21.428571  0.000494  17.135591   

         21        22        23        24        25        26        27  
0  0.327029  0.000563 -0.21212

In [45]:
# Split the input data and target values into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=42)

In [46]:
# Normalize the input data using a scaler like the StandardScaler from sklearn.preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [47]:
# Reshape the input data into a 3D tensor for use with the LSTM model
X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

### 4. Define and build the LSTM-model

In [48]:
# Create an instance of the Sequential model from Keras
lstm_model = Sequential()

In [49]:
# Add an LSTM layer with 64 units and input_shape=(1, X_train_scaled.shape[1])
lstm_model.add(LSTM(units=64, input_shape=(1, X_train_scaled.shape[1])))

In [50]:
# Add a dropout layer to avoid overfitting
lstm_model.add(Dropout(0.2))

In [51]:
# Add a dense layer with 3 units for the output
lstm_model.add(Dense(units=3))

In [52]:
# Compile the model using a mean squared error loss function and the Adam optimizer
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

### 5. Train the LSTM-model

In [53]:
# Train the model on the training dataset for 100 epochs
lstm_model.fit(X_train_reshaped, y_train, epochs=15, batch_size=32)

Epoch 1/15
4606/4606 [==============================] - 30s 6ms/step - loss: 0.0252
Epoch 2/15
4606/4606 [==============================] - 26s 6ms/step - loss: 8.8674e-04
Epoch 3/15
4606/4606 [==============================] - 29s 6ms/step - loss: 3.6910e-05
Epoch 4/15
4606/4606 [==============================] - 22s 5ms/step - loss: 2.2450e-05
Epoch 5/15
4606/4606 [==============================] - 25s 5ms/step - loss: 1.9961e-05
Epoch 6/15
4606/4606 [==============================] - 23s 5ms/step - loss: 1.9310e-05
Epoch 7/15
4606/4606 [==============================] - 20s 4ms/step - loss: 1.9059e-05
Epoch 8/15
4606/4606 [==============================] - 18s 4ms/step - loss: 1.9156e-05
Epoch 9/15
4606/4606 [==============================] - 19s 4ms/step - loss: 1.8815e-05
Epoch 10/15
4606/4606 [==============================] - 15s 3ms/step - loss: 1.8359e-05
Epoch 11/15
4606/4606 [==============================] - 18s 4ms/step - loss: 1.8674e-05
Epoch 12/15
4606/4606 [===========

### 6. Evaluate the trained model

In [54]:
# Use the trained model to make predictions on the testing dataset
y_pred = lstm_model.predict(X_test_reshaped)

1152/1152 [==============================] - 3s 2ms/step


In [55]:
# Create a new scaler for the output features
output_scaler = StandardScaler()
output_scaler.fit(target)
# Invert the scaling of the predictions and actual values
y_pred = output_scaler.inverse_transform(y_pred)
y_test = output_scaler.inverse_transform(y_test)

In [56]:
# Evaluate the performance of the model using metrics like mean squared error, mean absolute error, and R-squared
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [57]:
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 1.401944928539902e-08
Mean Absolute Error: 9.036521896306921e-05
R-squared: 0.9985770963901572


### 7. Save the model

In [58]:
# Save the model
lstm_model.save("lstm_model.h5")